In [ ]:
def max_drawdown (df, window=252):
    roll_max = df['close'].rolling(window, min_periods=1).max()
    daily_drawdown = df['close']/roll_max - 1.0
    max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()
    
    return max_daily_drawdown

In [ ]:
close_df = stock_data['NVDA']['close']
close = pd.DataFrame(close_df)
close.head()

In [ ]:
drawndown_data = (max_drawdown(close, window=252))
drawndown_data

In [ ]:
df = pd.DataFrame(drawndown_data)
df.plot()

In [3]:
import os
import numpy as np
import random
import datetime
import seaborn as sns
import pandas as pd
import yfinance as yf
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from scipy.interpolate import interp1d
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error


# Initialize python files and import functions
import stock_data as data
import algo_strategy as strategy
import nn_models as model
import backtesting as backtest

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'stock_data'

In [ ]:
def fetch_stock_data(start_date, end_date, tickers, timeframe='1Day'):
    
    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    api = REST(alpaca_api_key, alpaca_secret_key, base_url='https://paper-api.alpaca.markets')
    
    # Set start and end dates
    start_date_iso = pd.Timestamp(start_date, tz='America/New_York').isoformat()
    end_date_iso = pd.Timestamp(end_date, tz='America/New_York').isoformat()
    
    # pull data from Api
    df_ticker = api.get_bars(tickers, TimeFrame.Day, start=start_date_iso, end=end_date_iso).df
    dfs = {}
    
    for ticker in tickers:
        ticker_df = df_ticker.loc[df_ticker.symbol == ticker].drop('symbol', axis=1)
        ticker_df = ticker_df[['open', 'close', 'high', 'volume']]
        dfs[ticker] = ticker_df
        
    data = pd.concat(dfs.values(), axis=1, keys=tickers)
    data.reset_index(inplace=True)
    data.timestamp = data.timestamp.dt.date
    data.set_index('timestamp', inplace=True)
    data.index = pd.to_datetime(data.index)
    
    return data

In [ ]:
peroid =5 
df_stock['C'] = pd.Series(
    data=[df_stock['close'].iloc[:period].mean()],
    index=[df_stock['close'].index[period-1]],
).append(
    df['close'].iloc[period:]
).ewm(
    alpha=1.0 / period,
    adjust=False,
).mean()

In [ ]:
df_stock['WMA'] = pd.DataFrame(
    data=[df_stock['close'].iloc[:short_window].mean()],
    index=[df_stock['close'].index[short_window-1]],
).append(
    df['close'].iloc[short_window:]
).ewm(
    alpha=1.0 / period,
    adjust=False,
).mean()

In [ ]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df_stock, n=14):
    data = df_stock.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = df_stock['WMA'](tr, n)
    return atr